## Recomendador en Python

#### Kevin Craig Alisauskas


Importamos librerías

In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Importamos los datos

In [2]:
ratings = pd.read_csv("./Datos/ratings.dat", sep = "::", header = None, names = ["UserID", "MovieID", "Rating", "Timestamp"])
movies = pd.read_csv("./Datos/movies.dat", sep = "::", header = None, names = ["MovieID", "Title", "Genres"])

Vamos a crear dos matrices a partir de los usuarios, una con los nombres de las películas y si han visto la película o no (_ratings_matrix_bin_), a partir de la que realizaremos una recomendación por usuarios usando kneighbors, y otra con las valoraciones de las películas en lugar de si las han visto o no (_ratings_matrix_), para realizar una recomendación individual usando la correlación entre películas.

In [3]:
ratings_matrix = ratings.pivot_table(index = "UserID", columns = "MovieID", values = "Rating")
ratings_matrix_bin = np.where(np.isnan(ratings_matrix), 0, 1)

Nos conviene almacenar los vectores de usuarios y películas tal y como están.

In [4]:
ratings_matrix_cop = pd.DataFrame(ratings_matrix)
movie_id = ratings_matrix_cop.columns.values
users_id = ratings_matrix_cop.index

Seleccionamos el usuario para el cual recomendar. El índice será el índice matricial, necesitamos de _users_id_ para saber cual es en realidad.

In [5]:
user_id = 0

Realizamos el clústering por los vecinos más próximos y obtenemos las distancias y los 3 vecinos más cercanos (2 y el mismo usuario).

In [6]:
nbrs = NearestNeighbors(n_neighbors=3).fit(ratings_matrix_bin)
distances, indices = nbrs.kneighbors([ratings_matrix_bin[user_id]])

print("Distancias y vecinos al usuario con UserID = ", user_id, ":",  distances, indices)

Distancias y vecinos al usuario con UserID =  0 : [[0.         7.41619849 7.68114575]] [[   0 5342 2378]]


Seguiremos el siguiente criterio de recomendacion: el mas similar al preferido por el usuario y no escogido todavia. 
Para seguirlo necesitamos un listado de posibles películas. A partir del vecino más cercano obtenemos las películas que si ha visto ese vecino y no nuestro usuario.

In [8]:
difs = ratings_matrix_bin - ratings_matrix_bin[user_id]
vecino = indices[0, 1]   # 2ndo vecino, el primero es el mismo
posibles = (difs[vecino]  == 1)
Lposibles = movie_id[posibles]  # Lista de posibles películas a recomedar, sin valorar.

Observamos las películas vistas por el user_id y las posibles.

In [21]:
 
vistas = movie_id[np.where(ratings_matrix_bin[user_id] == 1)[0]]

print("Películas vistas por el usuario: ", users_id[user_id])
for i in vistas:
    print(movies.iloc[np.where(movies.MovieID == i)[0], 1].tolist()[0])

print("\n")
    
print("Películas Posibles: ")
for i in Lposibles:
    print(movies.iloc[np.where(movies.MovieID == i)[0], 1].tolist()[0])

Películas vistas por el usuario:  1
Toy Story (1995)
Pocahontas (1995)
Apollo 13 (1995)
Star Wars: Episode IV - A New Hope (1977)
Schindler's List (1993)
Secret Garden, The (1993)
Aladdin (1992)
Snow White and the Seven Dwarfs (1937)
Beauty and the Beast (1991)
Fargo (1996)
James and the Giant Peach (1996)
Wallace & Gromit: The Best of Aardman Animation (1996)
Close Shave, A (1995)
Hunchback of Notre Dame, The (1996)
My Fair Lady (1964)
Wizard of Oz, The (1939)
Gigi (1958)
Cinderella (1950)
Mary Poppins (1964)
Dumbo (1941)
Sound of Music, The (1965)
E.T. the Extra-Terrestrial (1982)
One Flew Over the Cuckoo's Nest (1975)
Princess Bride, The (1987)
To Kill a Mockingbird (1962)
Dead Poets Society (1989)
Back to the Future (1985)
Ben-Hur (1959)
Ponette (1996)
Hercules (1997)
Titanic (1997)
Last Days of Disco, The (1998)
Mulan (1998)
Rain Man (1988)
Driving Miss Daisy (1989)
Bambi (1942)
Saving Private Ryan (1998)
Antz (1998)
Pleasantville (1998)
Meet Joe Black (1998)
Bug's Life, A (1998)


Ahora vamos a trabajar con la matriz ratings_matrix, buscando la correlación entre películas a partir de las valoraciones de usuarios y mostramos la seleccionada (la que más se relacione con las películas vistas por el usuario al que recomendamos).

In [23]:

if len(Lposibles > 0):

    Lscores = []

    for p in Lposibles:

        # Guardamos la matriz de ratings de la p ésima película a recomendar.
        ratings_matrix_pos = ratings_matrix[p]

        # Guardamos la matriz de ratings de las películas vistas por el usuario UserID.
        ratings_matrix_user = ratings_matrix[vistas]

        # Calculamos la similitud entre la película posible y las vistas por el usuario.
        simil = pd.DataFrame(ratings_matrix_user.corrwith(ratings_matrix_pos))

        # Almacenamos la suma de las correlaciones de la posible película con las vistas.
        Lscores.append(simil.sum()[0])

    irecom = int(np.array(Lscores).argmax())
    print("Recomendacion Final: ")
    print(movies.iloc[np.where(movies.MovieID == Lposibles[irecom])[0], 1].tolist()[0])
    
else:
    print("Ninguna película diponible.")


Recomendacion Final: 
Lady and the Tramp (1955)
